In [1]:
cd /kaggle/input/trainXX

/kaggle/input/trainXX


In [2]:
path = '/kaggle/input/trainXX'

In [3]:
import numpy as np
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D,LeakyReLU,Conv2DTranspose
from keras.layers import Activation
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.initializers import RandomNormal
from keras.utils import to_categorical
from keras.layers import Input, Dense
from keras.layers import Concatenate

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,)
                                   
test_datagen = ImageDataGenerator(rescale = 1./255)

In [7]:
training_set = train_datagen.flow_from_directory(path,
                                                target_size=(28,56),
                                                class_mode = 'categorical',
                                                shuffle=True,color_mode='grayscale')

Found 60000 images belonging to 10 classes.


In [8]:

# define the discriminator model
def define_discriminator(image_shape=(28,28,1,)):
    in_image = Input(shape=image_shape)
    out_image = Input(shape=image_shape)
    init = RandomNormal(stddev=0.02)
    merged = Concatenate(axis=-1)([in_image, out_image])
    
    
    d = Conv2D(64, (3,3), kernel_initializer=init)(merged)
    d= LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (3,3), kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Dropout(0.5)(d)
    d = MaxPooling2D(pool_size=2,strides=2)(d)
  
    d = Conv2D(196, (3,3), kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Dropout(0.5)(d)
    d = MaxPooling2D(pool_size=2,strides=2)(d)
  
    d = Conv2D(256, (3,3), kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Dropout(0.5)(d)
    d = MaxPooling2D(pool_size=2,strides=2)(d)
  
    d = Flatten()(d)
    d = Dense(256, activation='relu')(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Dropout(0.5)(d)
  
    d = Dense(128)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Dropout(0.5)(d)
  
    d = Dense(64)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Dropout(0.5)(d)
    
    d = Dense(32)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = Dense(1)(d) 
    patch_out = Activation('sigmoid')(d)
  
    model = Model([in_image, out_image], patch_out)
    print('Compiling Discriminator')
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'], loss_weights=[0.5])
    model.summary()
    return model

In [9]:
def define_generator(image_shape=(28,28,1,)):
    in_image = Input(shape=image_shape)
    g = Conv2D(64, kernel_size=3, activation='relu',padding='same', input_shape=(28,28,1))(in_image)
    g = BatchNormalization()(g)
    
    g = Conv2D(128, kernel_size=3, strides=(2, 2))(g)
    g = BatchNormalization()(g)
    g = LeakyReLU(alpha=0.2)(g)
    g = Dropout(0.25)(g)
    
    g = Conv2D(196, kernel_size=3, strides=(2, 2))(g)
    g = BatchNormalization()(g)
    g = LeakyReLU(alpha=0.2)(g)
    g = Dropout(0.25)(g)
    
    g = Conv2D(256, kernel_size=3, strides=(2, 2))(g)
    g = BatchNormalization()(g)
    g = LeakyReLU(alpha=0.2)(g)
    g = Dropout(0.25)(g)
    
    g = Conv2D(512, kernel_size=2, strides=(2, 2))(g)
    g = BatchNormalization()(g)
    g = LeakyReLU(alpha=0.2)(g)
    g = Dropout(0.25)(g)
    
    g = Conv2DTranspose(512, kernel_size=3, strides=(2, 2))(g)
    g = BatchNormalization()(g)
    g = LeakyReLU(alpha=0.2)(g)
    g = Dropout(0.25)(g)
    
    g = Conv2DTranspose(256, kernel_size=2, strides=(2, 2))(g)
    g = BatchNormalization()(g)
    g = LeakyReLU(alpha=0.2)(g)
    g = Dropout(0.25)(g)
    
    g = Conv2DTranspose(128, kernel_size=3, strides=(2, 2))(g)
    g = BatchNormalization()(g)
    g = LeakyReLU(alpha=0.2)(g)
    g = Dropout(0.25)(g)
    
    g = Conv2DTranspose(1, kernel_size=4, strides=(2, 2))(g)
    out_image = Activation('tanh')(g)
    model = Model(in_image, out_image)
    print('Compiling generator')
    model.summary()
    return model

In [10]:
discriminator = define_discriminator()

Compiling Discriminator
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 28, 28, 2)    0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 26, 26, 64)   1216        con

In [11]:
generator = define_generator()

Compiling generator
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_8 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
batch_normalization_9 (Batch (None, 13, 13, 128)       512       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 13, 13, 128)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 13,

In [12]:
def define_gan(g_model, d_model, image_shape=(28,28,1,)):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    # connect the source image to the generator input
    gen_out = g_model(in_src)
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out])
    # src image as input, generated image and classification output
    model = Model(in_src, [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [13]:
def generate_fake_samples(g_model, image_hand, patch_shape=1):
    # generate fake instance
    X = g_model.predict(image_hand)
    # create 'fake' class labels (0)
    y = np.zeros((len(X), patch_shape))
    return X, y

In [14]:
def generat_real_samples( patch_shape=1):
    X,y = training_set.next()
    real_print,real_hand = X[:,:,:28],X[:,:,28:]
    real_hand = np.reshape(real_hand,[-1, 28, 28, 1])
    real_print = np.reshape(real_print,[-1, 28, 28, 1])
    real_label = np.ones((len(X), patch_shape))
    return [real_hand,real_print],real_label

In [15]:
I = training_set.n//training_set.batch_size
I

1875

In [16]:
gan_model = define_gan(generator,discriminator)

In [17]:
for i in range(I):
    #generating real sample
    [real_hand,real_print],real_label = generat_real_samples()
    #generating fake sample
    fake_print,fake_label = generate_fake_samples(generator,real_hand)
    #update discriminator for real sample
    d_loss1 = discriminator.train_on_batch([real_hand,real_print,], real_label)
    #update discriminator for fake sample
    d_loss2 = discriminator.train_on_batch([fake_print,real_print,], fake_label)
    # update the generator
    g_loss, _, _ = gan_model.train_on_batch(real_hand, [real_label, real_print])
    #print performance
    print("epoch {}: real_loss: {}---fake_loss: {}---GAN_LOSS: {}".format(i+1, d_loss1,d_loss2,g_loss))
    if i % 10 == 0:
        #filename = '/GAN/model_%06d' % (i+1)
        #g_model.save(filename+'gan.h5')
        #generator.save(filename+'generator.h5')
        #discriminator.save(filename+'discriminator.h5')
        a = np.random.randint(1,32)
        i = real_print[a]
        j = np.reshape(i,(28,28))
        plt.subplot(1,3,1)
        plt.imshow(j,cmap='gray')
        i = real_hand[a]
        j = np.reshape(i,(28,28))
        plt.subplot(1,3,2)
        plt.imshow(j,cmap='gray')
        i = fake_print[a]
        j = np.reshape(i,(28,28))
        plt.subplot(1,3,3)
        plt.imshow(j,cmap='gray')
        plt.show()
        

/opt/conda/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


NameError: name 'fake_hand' is not defined